In [1]:
from IPython.display import display
import usdaquickstats
import pandas as pd
import numpy as np
import io
import os
os.environ["USDA_API_KEY"]

KeyError: 'USDA_API_KEY'

First we grab some data from the API using the usdaquickstats.py module

In [2]:
u = usdaquickstats.USDAQuickStats()
u.getparams["year"] = 2012
u.getparams["state_alpha"] = ("MO","TX")
u.getparams["commodity_desc"] = "CORN"
u.get()

KeyError: 'USDA_API_KEY'

In [ ]:
pd.set_option('display.max_row',100) #set the default number of rows to display when printing the df
pd.set_option('display.max_columns',50) #set the default max number of columns to display when printing the df
df = pd.read_csv(io.StringIO(u.response.content.decode('utf-8')))

Next we'll try summarizing some data in a pandas dataframe

http://bconnelly.net/2013/10/summarizing-data-in-python-with-pandas/

Let's first take a look at the columns and data types in the data frame.

Now we can create a groupbyobject and produce some summaries for those groups.

First we need to create a numeric version of the Value field since the original field contains string values for the suppressed cells i.e. (D). Because the original column is an 'object' (string) data type, values that included commas for the million and thousand delimiters are stored as, for example, "1,000" instead of "1000". 

So we use coerce to convert values to numbers when possible.

In [ ]:
df['ValueNum'] = pd.to_numeric(df['Value'].str.replace(",",""),errors='coerce')

groups = df[df.short_desc.str.contains('OPERATIONS')].groupby(['state_alpha','short_desc'])

pd.set_option('display.float_format',lambda x: '%.3f' % x)

dfg = groups['ValueNum'].aggregate([np.mean,np.sum,np.count_nonzero])

dfg

If we want to see the distinct values in a column, we use df.col_name.unique() (so easy!)

Now I want to get the rows from the dataframe where the county_name is NaN... I suspect these are sub-totals at
at state or some other non-county level.

In [ ]:
dfcounty = df[pd.notnull(df.county_name)]
dfharv = dfcounty[dfcounty.short_desc.str.contains('ACRES HARVESTED')]

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
# groups = dfharv[dfharv.state_alpha=='MO'].groupby('short_desc')
# moharv = groups['ValueNum'].aggregate([np.sum])

# groups = dfharv[dfharv.state_alpha=='TX'].groupby('short_desc')
# txharv = groups['ValueNum'].aggregate([np.sum])

# modat = go.Bar(x=moharv.index,y=moharv['sum'],name='Missouri')
# txdat = go.Bar(x=txharv.index,y=txharv['sum'],name='Texas')

# gdat = [modat, txdat]
# layout = go.Layout(
#     barmode='group'
# )

# py.iplot(gdat,filename = 'pandas-bar-chart')

In [27]:
# create a DataFrameGroupBy object
gbo = dfharv.groupby(['state_alpha','short_desc'])
display(gbo)
# type(gbo)

# we can group a groupby object if using a multi-index (i.e. more than one grouping column, as above)
stategbo = gbo.groupby(level=0)
# or
stategbo = gbo.groupby('state_alpha')
display(stategbo)

# this is a series object
sgbo = gbo['ValueNum'].sum()
display(sgbo)
# type(sgbo)

# using .aggregate returns a dataframe ¯\_(ツ)_/¯
sumgbo = gbo['ValueNum'].aggregate([np.sum])
display(sumgbo)
# type(sumgbo)



# sumgbo.groupby(level=0).sum()
# harv = groups['ValueNum'].aggregate([np.sum])
# harv.filter['state_alpha=''MO''']
#harv['MO']
#modat = go.Bar(x=moharv.index,y=moharv['sum'],name='Missouri')
#txdat = go.Bar(x=txharv.index,y=txharv['sum'],name='Texas')

#gdat = [modat, txdat]
#layout = go.Layout(
#    barmode='group'
#)

#py.iplot(gdat,filename = 'pandas-bar-chart')

AttributeError: Cannot access callable attribute 'groupby' of 'DataFrameGroupBy' objects, try using the 'apply' method